In [1]:
import os
import yaml
import logging
import numpy as np
import importlib
import subprocess
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Any, Optional, Tuple, Dict
from datetime import datetime
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr, kurtosis, skew

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [5]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [6]:
# Текущая рабочая директория
cwd = Path().resolve()

# Поднимаемся на один уровень выше
project_root = cwd.parent

# Добавляем корень проекта в sys.path
sys.path.append(str(project_root))

# Загрузка данных из config.yaml
from src.utils import ml_utils, eda_utils

# Путь к файлу config.yaml
config_path = project_root / "config" / "config.yaml"

# Загружаем конфиг
config = ml_utils.load_config(config_path)

## 3.1. Загрузка данных

In [7]:
# Загрузка train и test
df_train = ml_utils.data_load(data_type='train', config=config)
df_test = ml_utils.data_load(data_type='test', config=config)

In [8]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Id,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,230,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28
1,231,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59
2,232,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82
3,233,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76
4,234,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40


In [9]:
# Вывод первых 5 строк тестового датасета
df_test.head()

,Id,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,0,167.4,129.9,128.6,175.5,7.8,1006.3,746.6,28
1,1,475.0,118.8,0.0,181.1,8.9,852.1,781.5,7
2,2,251.4,0.0,118.3,188.5,6.4,1028.4,757.7,100
3,3,307.0,0.0,0.0,193.0,0.0,968.0,812.0,365
4,4,143.6,0.0,174.9,158.4,17.9,942.7,844.5,28


In [10]:
# Удаление неинформативного признака Id
df_train = df_train.drop(columns=["Id"])
df_test = df_test.drop(columns=["Id"])

In [11]:
# Удаление дубликатов
df_train_cleaned = df_train.drop_duplicates()
test_cleaned = df_test.drop_duplicates()

## 3.2. Создание инженерных признаков

In [12]:
# Создание признаков (w_c, SP_pct)
df_train_feat = eda_utils.add_concrete_ratios(df_train_cleaned)
df_test_feat = eda_utils.add_concrete_ratios(df_test)

df_train_feat.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923


## 3.3. Оценка объема данных и выбор алгоритмов

In [13]:
# Формируем список алгоритмов исходя из объема данных по правилу NEPV
models = ml_utils.check_models_by_nepv(df_train_feat, config)

[REGRESSION] Правило NEPV: 781 наблюдений / 11 признаков = 71.00
LinearRegression / Ridge / Lasso — соответствуют (≥ 20)
CHAID (не реализован в sklearn) — соответствует (≥ 50)
Сложные модели (RF, Boosting и др.) — не соответствуют (< 200)
DecisionTreeRegressor (CART) — добавлен с осторожностью. NEPV к нему не применяется строго.

Список рекомендованных моделей: ['LinearRegression', 'Ridge', 'Lasso', 'DecisionTreeRegressor']


## 3.4. Проверка порядка признаков в тренировочном и тестовом датасете

In [14]:
# Создание массива из признаков и массива из целевой переменной
X = df_train_feat.drop(columns=["Strength"])
y = df_train_feat["Strength"]

In [15]:
# Сравниваем порядок признаков в тренировочном и тестовом датасете
if list(X.columns) == list(df_test_feat.columns):
    print("Порядок признаков совпадает")
else:
    print("Порядок признаков отличается")

Порядок признаков совпадает


## 3.5. Генерация EDA отчета

#### Предварительный отчет

In [ ]:
# Общая информация
df_train_feat.info()

In [ ]:
# Проверка на пропуски train
df_train_feat.isna().sum()

In [ ]:
# Проверка на пропуски test
df_train_feat.isna().sum()

In [ ]:
# Основные статистики train
df_train_feat.describe()

#### Генерация подробного отчета ydataprofaling

In [ ]:
# Генерация отчета по тренировочным данным с помощью ydata-profiling
ml_utils.eda_report(df_train_feat, "train", config)

## 3.6. Анализ выбросов

#### Анализ выбросов

In [ ]:
# Построение сводной таблицы по выбросам на оснвове IQR и значений ГОСТ
summary_df,  outlier_masks_df = eda_utils.detect_outliers(df_train_feat, config)
summary_df

In [ ]:
# Построение графиков 
ml_utils.plot_outliers(df_train_feat, summary_df, max_plots=10)

In [ ]:
# Сохранение отчетов
output_dir=Path().resolve().parent / config["output"]["eda_report_dir"]

ml_utils.save_outliers_report(summary_df, output_dir=output_dir)

## 3.7. Анализ нулей в признаках

#### Анализ признаков с нулнвыми значениями

In [ ]:
config = eda_utils.analyze_zeros(df_train_feat, config)

## 3.8. Анализ зависимости целевой переменнной от признаков

### Определение функций

In [ ]:
def visualize_feature_analysis(analysis_df: pd.DataFrame) -> None:
    """
    Визуализирует сравнение корреляций Пирсона и Спирмена для каждого признака с целевой переменной и
    значения асимметрии и эксцесса для признаков.
    
    Parameters:
    -----------
    analysis_df : pandas.DataFrame
        Датафрейм с результатами анализа признаков, содержащий колонки:
        - feature: названия признаков
        - pearson_corr: коэффициенты корреляции Пирсона
        - spearman_corr: коэффициенты корреляции Спирмена  
        - skewness: значения асимметрии
        - kurtosis: значения эксцесса
    
    Returns:
    --------
    None
        Функция отображает графики и не возвращает значений
    """
    
    plt.figure(figsize=(15, 6))
    
    # График 1: Сравнение корреляций
    plt.subplot(1, 2, 1)
    indices = range(len(analysis_df))
    width = 0.35
    plt.bar([i - width/2 for i in indices], analysis_df['pearson_corr'], 
            width, label='Pearson', alpha=0.7)
    plt.bar([i + width/2 for i in indices], analysis_df['spearman_corr'], 
            width, label='Spearman', alpha=0.7)
    plt.xticks(indices, analysis_df['feature'], rotation=90, ha='right')
    plt.legend()
    plt.title('Сравнение корреляций Пирсона и Спирмена')
    plt.grid(True, alpha=0.3)
    
    # График 3: Распределение признаков (skewness + kurtosis)
    plt.subplot(1, 2, 2)
    plt.scatter(analysis_df['skewness'], analysis_df['kurtosis'], 
               s=100, alpha=0.7)
    for i, row in analysis_df.iterrows():
        plt.annotate(row['feature'], (row['skewness'], row['kurtosis']),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    plt.axvline(0, color='gray', linestyle='--', alpha=0.5)
    plt.axhline(0, color='gray', linestyle='--', alpha=0.5)
    plt.xlabel('Skewness')
    plt.ylabel('Kurtosis')
    plt.title('Распределение признаков (skewness vs kurtosis)')
    plt.grid(True, alpha=0.3)    
    
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_feature_trends(df, features, target='Strength', figsize=(16, 12), alpha=0.2):
    """
    Строит scatter plot для каждого признака с наложением различных трендов.
    Возвращает датасет с лучшими преобразованиями и R² scores.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Исходный датафрейм с данными
    features : list
        Список признаков для анализа
    target : str
        Название целевой переменной
    figsize : tuple
        Размер figure для plotting
    alpha : float
        Пороговое значение для выбора линейного тренда (0.2 = 20%)
    """
    
    # Определяем функции для трендов
    def linear_func(x, a, b):
        return a * x + b
    
    def log_func(x, a, b):
        return a * np.log(x + 1e-10) + b
    
    def sqrt_func(x, a, b):
        return a * np.sqrt(x) + b
    
    def reciprocal_func(x, a, b):
        return a / (x + 1e-10) + b
    
    def square_func(x, a, b):
        return a * (x ** 2) + b
    
    # Создаем grid subplots
    n_features = len(features)
    n_cols = 2
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()
    
    # Цвета для разных трендов
    colors = ['red', 'blue', 'green', 'orange', 'purple']
    trend_names = ['Linear', 'Log', 'Sqrt', '1/x', 'x²']
    trend_funcs = [linear_func, log_func, sqrt_func, reciprocal_func, square_func]
    
    # Создаем датасет для результатов
    results_data = []
    
    for i, feature in enumerate(features):
        if i >= len(axes):
            break
            
        ax = axes[i]
        x_data = df[feature].values
        y_data = df[target].values
        
        # Scatter plot
        ax.scatter(x_data, y_data, alpha=0.6, s=30, color='gray', label='Data')
        ax.set_xlabel(feature)
        ax.set_ylabel(target)
        ax.set_title(f'{feature} vs {target}')
        
        # Сортируем данные для построения трендов
        sort_idx = np.argsort(x_data)
        x_sorted = x_data[sort_idx]
        y_sorted = y_data[sort_idx]
        
        # Строим различные тренды
        r2_scores = {}
        
        for j, (func, color, name) in enumerate(zip(trend_funcs, colors, trend_names)):
            try:
                # Подбираем параметры для функции тренда
                popt, _ = curve_fit(func, x_sorted, y_sorted, maxfev=5000)
                
                # Предсказываем значения для тренда
                y_trend = func(x_sorted, *popt)
                
                # Вычисляем R² score
                r2 = r2_score(y_sorted, y_trend)
                r2_scores[name] = r2
                
                # Строим тренд
                ax.plot(x_sorted, y_trend, color=color, linewidth=2, 
                       label=f'{name} (R²={r2:.3f})', alpha=0.8)
                
            except Exception as e:
                # Пропускаем тренды, которые не удается построить
                r2_scores[name] = -np.inf
                continue
        
        # Определяем лучшее преобразование
        best_trend_name = max(r2_scores.items(), key=lambda x: x[1])[0]
        best_r2 = r2_scores[best_trend_name]
        linear_r2 = r2_scores.get('Linear', -np.inf)
        
        # Если линейный тренд хуже лучшего менее чем на alpha, выбираем линейный
        if best_trend_name != 'Linear' and linear_r2 >= best_r2 * (1 - alpha):
            final_best_trend = 'Linear'
            final_best_r2 = linear_r2
        else:
            final_best_trend = best_trend_name
            final_best_r2 = best_r2
        
        # Добавляем данные в результаты
        results_data.append({
            'feature': feature,
            'best_transformation': final_best_trend,
            'best_r2_score': final_best_r2,
            'linear_r2_score': linear_r2,
            'log_r2_score': r2_scores.get('Log', np.nan),
            'sqrt_r2_score': r2_scores.get('Sqrt', np.nan),
            'reciprocal_r2_score': r2_scores.get('1/x', np.nan),
            'square_r2_score': r2_scores.get('x²', np.nan)
        })
        
        # Добавляем легенду
        ax.legend(loc='best', fontsize=8)
        
        # Добавляем аннотацию с лучшим трендом
        ax.annotate(f'Best: {final_best_trend} (R²={final_best_r2:.3f})',
                   xy=(0.05, 0.95), xycoords='axes fraction',
                   fontsize=9, bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8))
        
        # Добавляем корреляции в заголовок
        pearson_corr = np.corrcoef(x_data, y_data)[0, 1]
        spearman_corr = pd.Series(x_data).corr(pd.Series(y_data), method='spearman')
        ax.set_title(f'{feature} vs {target}\nPearson: {pearson_corr:.3f}, Spearman: {spearman_corr:.3f}')
    
    # Скрываем пустые subplots
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Создаем и возвращаем датасет с результатами
    results_df = pd.DataFrame(results_data)
    return results_df

### Анализ корреляции целевой переменнной и признаков

In [ ]:
# Создает комплексный датафрейм c результатами анализа
# корреляции признаков и целевой переменной
df_corr_target = ml_utils.create_feature_analysis(X, y)
df_corr_target

**Выводы:** Признак Fly Ash показывает очень слабую и статистически незначимую связь с целевой переменной Strength. Поэтому его стоит объединить с другими признаками или удалить

In [ ]:
# Визуализация анализа признаков
ml_utilsюvisualize_feature_analysis(df_corr_target) 

In [ ]:
results = plot_feature_trends(df_train_feat, features, target='Strength', figsize=(16, 50), alpha=0.25)
results